In [28]:
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
import os
import statistics
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

In [3]:
benign_path = '/home/sdsra/Downloads/CTU-Dataset/benign'
benign_datas = pd.DataFrame()
sub_dir_list = os.listdir(benign_path)
for dir in sub_dir_list:
    tmp_data = pd.read_pickle(benign_path + '/' + dir + '/datas-by-origin-tool.pkl')
    benign_datas = pd.concat([benign_datas, tmp_data], ignore_index=True)

In [5]:
benign_datas['is_malware'] = 0

In [6]:
malware_path = '/home/sdsra/Downloads/CTU-Dataset/malware-2' 
malware_datas = pd.DataFrame()
for dir in os.listdir(malware_path) :
    pkl_path = malware_path +'/'+dir+'/datas-by-origin-tool.pkl'
    if os.path.exists(pkl_path):
        tmp_data = pd.read_pickle(pkl_path)
        malware_datas = pd.concat([malware_datas, tmp_data], ignore_index=True)

In [7]:
malware_datas = malware_datas[malware_datas['label'] == 'botnet']

In [8]:
malware_datas['is_malware'] = 1

In [9]:
len(benign_datas), len(malware_datas)

(12971, 71409)

In [11]:
# malware_datas = malware_datas[malware_datas['label'] == 'botnet']
upsampled_benign_datas = benign_datas.sample(len(malware_datas), replace=True)

In [13]:
upsampled_benign_datas['is_malware'] = 0

In [14]:
datas = pd.concat([upsampled_benign_datas, malware_datas], ignore_index=True)

In [27]:
datas = pd.concat([benign_datas, malware_datas], ignore_index=True)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [15]:
numeric_columns = list(filter(lambda x: x not in ['src_ip','src_port','dst_ip','dst_port','proto', 'label'], benign_datas.columns))

In [16]:
datas['is_malware'].value_counts()

1    71409
0    71409
Name: is_malware, dtype: int64

In [17]:
X_train, X_test, y_train, y_test = train_test_split(datas, datas['is_malware'], test_size=0.33, random_state=42)

In [18]:
cls = RandomForestClassifier()

In [19]:
score = cross_validate(cls, datas[numeric_columns], datas['is_malware'], cv=10, return_train_score=False)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarni

In [20]:
np.mean(score['test_score'])

1.0

In [24]:
cls.fit(X_train[numeric_columns], y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
y_pred = cls.predict(X_test[numeric_columns])

In [30]:
cm = confusion_matrix(y_test, y_pred)

In [41]:
TN, FP, FN, TP = cm.ravel()

In [42]:
cm

array([[23537,     0],
       [    1, 23592]])

In [43]:
TN, FP

(23537, 0)

In [44]:
FN,TP

(1, 23592)

In [45]:
accuracy = (TN + TP) / (TN + TP + FP + FN)

In [46]:
precision = TP / (TP + FN)

In [47]:
print('accuracy : {}, precision: {}'.format(accuracy, precision))

accuracy : 0.9999787820920857, precision: 0.9999576145466875
